In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats import weightstats
pd.options.display.float_format = '{:.5f}'.format

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
senate = pd.read_csv('senate_returns.csv')
house = pd.read_csv('house_returns.csv')

In [3]:
write_to_csv = False

if write_to_csv:
    open('ws_returns.csv', 'w').close()

In [4]:
return_cols = ['return_abnormal_prev1m', 'return_abnormal_prev5d', 'return_abnormal_prev1d', 'return_abnormal_1d', 'return_abnormal_5d', 'return_abnormal_1m']

def winsorize_returns(returns, tr_type=None, display_df=False):
    returns = returns.dropna(subset=return_cols)
    if tr_type:
        returns = returns[returns['type'] == tr_type]
    
    if display_df:
        display(returns[return_cols].describe())
    
    winsorized_returns = returns.copy()
    
    winsorized_returns[return_cols] = returns[return_cols].apply(lambda col: stats.mstats.winsorize(col, limits=[0.01, 0.01]).data)
    if display_df:
        display(winsorized_returns[return_cols].describe())
        return
    
    return winsorized_returns

In [5]:
def summary(returns, name=None):
    df = returns.describe().loc[['count', 'mean', 'std', 'min', 'max']]
    
    ttest = returns.apply(lambda col: stats.ttest_1samp(col, 0))   
    df = df.append(ttest)
    df = df.rename({0: 't-stat', 1: 'p-value'})
    df = df.rename(lambda x: x.split('_')[2], axis=1)
    
    if write_to_csv and name:
        with open('ws_returns.csv', 'a') as f:
            f.write('\n\n{}\n'.format(name))
            df.to_csv(f)
        
    return df

In [6]:
summary(winsorize_returns(house, tr_type='Purchase')[return_cols], 'House Purchase')

,prev1m,prev5d,prev1d,1d,5d,1m
count,2658.00000,2658.00000,2658.00000,2658.00000,2658.00000,2658.00000
mean,-0.00308,0.00472,0.00028,-0.00024,0.00369,0.00022
std,0.13102,0.07448,0.03139,0.03067,0.06842,0.12667
min,-0.40240,-0.24791,-0.11030,-0.10671,-0.18022,-0.38918
max,0.44066,0.27152,0.10207,0.11526,0.27039,0.44609
t-stat,-1.21134,3.26459,0.45985,-0.41034,2.77896,0.09141
p-value,0.22587,0.00111,0.64566,0.68159,0.00549,0.92717


In [7]:
summary(winsorize_returns(house, tr_type='Sale')[return_cols], 'House Sale')

,prev1m,prev5d,prev1d,1d,5d,1m
count,2489.00000,2489.00000,2489.00000,2489.00000,2489.00000,2489.00000
mean,-0.00948,0.00163,0.00091,-0.00290,-0.01334,-0.02627
std,0.16143,0.09063,0.03336,0.03501,0.08619,0.18456
min,-0.62695,-0.41773,-0.11672,-0.12778,-0.40620,-0.97060
max,0.38611,0.28953,0.11788,0.10312,0.20961,0.32676
t-stat,-2.93113,0.89704,1.36062,-4.13882,-7.72380,-7.10220
p-value,0.00341,0.36979,0.17376,0.00004,0.00000,0.00000


In [8]:
summary(winsorize_returns(senate, tr_type='Purchase')[return_cols], 'Senate Purchase')

,prev1m,prev5d,prev1d,1d,5d,1m
count,536.00000,536.00000,536.00000,536.00000,536.00000,536.00000
mean,-0.00975,-0.00748,-0.00101,-0.00180,-0.00002,-0.01141
std,0.17477,0.08461,0.03730,0.03198,0.08159,0.15129
min,-0.75026,-0.31583,-0.12679,-0.10760,-0.24778,-0.72906
max,0.53339,0.27295,0.13456,0.11122,0.36034,0.36926
t-stat,-1.29153,-2.04622,-0.62695,-1.30563,-0.00642,-1.74623
p-value,0.19708,0.04122,0.53096,0.19224,0.99488,0.08134


In [9]:
summary(winsorize_returns(senate, tr_type='Sale')[return_cols], 'Senate Sale')

,prev1m,prev5d,prev1d,1d,5d,1m
count,605.00000,605.00000,605.00000,605.00000,605.00000,605.00000
mean,0.00670,0.00488,0.00433,0.00018,0.00135,0.01468
std,0.14990,0.07377,0.03071,0.03079,0.06465,0.14237
min,-0.43556,-0.20249,-0.09702,-0.11499,-0.22090,-0.36836
max,0.45439,0.31793,0.09236,0.07105,0.20835,0.48313
t-stat,1.09954,1.62792,3.47072,0.14498,0.51500,2.53609
p-value,0.27197,0.10406,0.00056,0.88478,0.60674,0.01146


In [10]:
def avg_by_stock(returns, tr_type=None):
    ws_returns = winsorize_returns(returns, tr_type)
    return ws_returns.groupby('ticker')[return_cols].mean()

In [11]:
# summary(avg_by_stock(house, tr_type='Purchase'))

In [12]:
# summary(avg_by_stock(house, tr_type='Sale'))

In [13]:
# summary(avg_by_stock(senate, tr_type='Purchase'))

In [14]:
# summary(avg_by_stock(senate, tr_type='Sale'))

In [15]:
amount_midpoint_map = {
    '$1,001 -':1000,
    '$1,001 - $15,000': 8000.5,
    '$15,001 - $50,000': 32500.5,
    '$50,001 - $100,000': 75000.5,
    '$100,001 - $250,000':125000.5,
    '$250,001 - $500,000':375000.5,
    '$500,001 - $1,000,000':750000.5,
    '$1,000,001 - $5,000,000':2500000.5,
    '$5,000,001 - $25,000,000':15000000.5,
}

def weight_by_amount(returns, filename, tr_type=None):
    ws_returns = winsorize_returns(returns, tr_type)
    weights = ws_returns['amount'].map(lambda a: np.log(amount_midpoint_map[a]))
    weighted_returns = ws_returns.copy()
    weighted_returns[return_cols] = weighted_returns[return_cols].apply(lambda col: col * weights)
    weighted_returns.to_csv(filename)

def weight_by_amount_summary(returns, tr_type=None):
    ws_returns = winsorize_returns(returns, tr_type)
    weights = ws_returns['amount'].map(lambda a: np.log(amount_midpoint_map[a]))
    count = [len(ws_returns)] * 6
    mean = weightstats.DescrStatsW(ws_returns[return_cols], weights=weights).mean
    ttest = ws_returns[return_cols].apply(lambda col: weightstats.ttest_ind(col, [0]*len(col), weights=(weights, None)))
    df = pd.DataFrame(columns=return_cols)
    df.loc['count'] = count
    df.loc['mean'] = mean
    df = df.append(ttest)
    df = df.rename({0: 't-stat', 1: 'p-value'})
    df = df.rename(lambda x: x.split('_')[2], axis=1) 
    return df

In [16]:
weight_by_amount(house, 'ws_weighted_house_returns.csv')
weight_by_amount(senate, 'ws_weighted_senate_returns.csv')

In [17]:
weight_by_amount_summary(house, tr_type='Purchase')

,prev1m,prev5d,prev1d,1d,5d,1m
count,2658,2658,2658,2658,2658,2658
mean,-0.00268,0.00469,0.00031,-0.00039,0.00344,0.00012
t-stat,-1.05191,3.26395,0.51177,-0.66263,2.62057,0.04855
p-value,0.29285,0.0011,0.60881,0.50757,0.00878,0.96128
2,27503.70921,27503.70921,27503.70921,27503.70921,27503.70921,27503.70921


In [18]:
weight_by_amount_summary(house, tr_type='Sale')

,prev1m,prev5d,prev1d,1d,5d,1m
count,2489,2489,2489,2489,2489,2489
mean,-0.00965,0.0013,0.00073,-0.00288,-0.01248,-0.02274
t-stat,-3.03326,0.72744,1.0923,-4.12084,-7.40551,-6.47964
p-value,0.00242,0.46697,0.27471,0.00004,0.0,0.0
2,25566.64568,25566.64568,25566.64568,25566.64568,25566.64568,25566.64568


In [19]:
weight_by_amount_summary(senate, tr_type='Purchase')

,prev1m,prev5d,prev1d,1d,5d,1m
count,536,536,536,536,536,536
mean,-0.00746,-0.00691,-0.00092,-0.00173,0.00048,-0.01034
t-stat,-0.98204,-1.89465,-0.57582,-1.25947,0.13602,-1.58215
p-value,0.32612,0.05819,0.56476,0.20791,0.89181,0.11367
2,5783.51913,5783.51913,5783.51913,5783.51913,5783.51913,5783.51913


In [20]:
weight_by_amount_summary(senate, tr_type='Sale')

,prev1m,prev5d,prev1d,1d,5d,1m
count,605,605,605,605,605,605
mean,0.00596,0.00484,0.00436,0.00063,0.00171,0.01296
t-stat,0.98626,1.62048,3.51688,0.51355,0.65812,2.28041
p-value,0.32404,0.10518,0.00044,0.60758,0.51048,0.02261
2,6738.95656,6738.95656,6738.95656,6738.95656,6738.95656,6738.95656


In [21]:
def avg_by_person(returns, person_col, tr_type=None):
    ws_returns = winsorize_returns(returns, tr_type)
    return ws_returns.groupby(person_col)[return_cols].mean()

In [22]:
# summary(avg_by_person(house, 'representative', tr_type='Purchase'))

In [23]:
# summary(avg_by_person(house, 'representative', tr_type='Sale'))

In [24]:
# summary(avg_by_person(senate, 'senator', tr_type='Sale'))

In [25]:
# house_daily = pd.read_csv('house_daily_returns.csv', index_col = 0)
# senate_daily = pd.read_csv('senate_daily_returns.csv', index_col = 0)
# senate_daily

In [26]:
# daily_returns_cols = [str(x) for x in range(-21,23)]

# def winsorize_daily_returns(daily_returns, tr_type):
#     ws_daily = daily_returns[daily_returns['type'] == tr_type]    
#     ws_daily[daily_returns_cols] = ws_daily[daily_returns_cols].apply(lambda col: stats.mstats.winsorize(col, limits=[0.01, 0.01]).data)
#     return ws_daily
    

In [27]:
# def cumulative_returns(daily_returns):
#     daily = daily_returns[daily_returns_cols]
#     cumulative = (daily+1).cumprod(axis=1)
#     return cumulative.mean() - 1

# hreturns = pd.DataFrame({"Purchase": cumulative_returns(winsorize_daily_returns(house_daily, 'Purchase')), "Sale": cumulative_returns(winsorize_daily_returns(house_daily, 'Sale'))})
# sreturns = pd.DataFrame({"Purchase": cumulative_returns(winsorize_daily_returns(senate_daily, 'Purchase')), "Sale": cumulative_returns(winsorize_daily_returns(senate_daily, 'Sale'))})

In [28]:
# def graph(hreturns, sreturns):
#     fig = make_subplots(rows = 1, cols = 2, subplot_titles=('House', 'Senate'), shared_yaxes = True)

#     fig.add_trace(go.Scatter(x = hreturns.index, y = hreturns['Purchase'], name = 'Purchase', legendgroup = 'Purchase', line_color = 'blue'), row = 1, col = 1)
#     fig.add_trace(go.Scatter(x = hreturns.index, y = hreturns['Sale'], name = 'Sale', legendgroup = 'Sale', line_color = 'red'), row = 1, col = 1)
#     fig.add_trace(go.Scatter(x = sreturns.index, y = sreturns['Purchase'], name = 'Purchase', showlegend = False, legendgroup = 'Purchase', line_color = 'blue'), row = 1, col = 2)
#     fig.add_trace(go.Scatter(x = sreturns.index, y = sreturns['Sale'], name = 'Sale', showlegend = False, legendgroup = 'Sale', line_color = 'red'), row = 1, col = 2)
#     fig.show()

In [29]:
# graph(hreturns, sreturns)